## change just here the sequence length that you want 

In [12]:
sequence_length_left = 0
sequence_length_right = 8750



# Les deux listes
liste_nc = [
    'NC_000001.11', 'NC_000002.12', 'NC_000003.12', 'NC_000004.12', 'NC_000005.10',
    'NC_000006.12', 'NC_000007.14', 'NC_000008.11', 'NC_000009.12',
    'NC_000010.11', 'NC_000011.10', 'NC_000012.12', 'NC_000013.11',
    'NC_000014.9', 'NC_000015.10', 'NC_000016.10', 'NC_000017.11',
    'NC_000018.10', 'NC_000019.10', 'NC_000020.11', 'NC_000021.9',
    'NC_000022.11', 'NC_000023.11', 'NC_000024.10'
]

liste_chr = [
    'chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8',
    'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16',
    'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22', 'chrX',
    'chrY'
]

# Création du dictionnaire dans l'ordre demandé
dex_to_ncbi = dict(zip(liste_chr, liste_nc))

import pandas as pd
from Bio import SeqIO

# Charger le fichier FASTA et stocker les séquences dans un dictionnaire
fasta_file = "../ncbi_dataset/data/GCF_000001405.40/GCF_000001405.40_GRCh38.p14_genomic.fna"
chromosome_sequences = {}
for record in SeqIO.parse(fasta_file, "fasta"):
    chromosome_sequences[record.id] = str(record.seq)

# Charger le DataFrame
df = pd.read_csv("dex_annotations_important.csv")
df_filtered = df[df['seqname'] != 'chrM']

# Définir la longueur de la séquence à extraire (par exemple, 5000 bases de chaque côté)

# Fonction pour extraire la séquence centrée autour de 'start' ou 'end' basé sur 'strand'
def get_centered_sequence(row):
    chromosome = dex_to_ncbi[row['seqname']]
    if row['strand'] == '+':
        center = row['start']
    else:
        center = row['end']
    
    seq = chromosome_sequences.get(chromosome, "")
    if seq:
        start_idx = max(0, center - sequence_length_left)
        end_idx = min(len(seq), center + sequence_length_right)
        return seq[start_idx-1:end_idx]
    return ""

# Ajouter la nouvelle colonne avec les séquences extraites
df_filtered['centred_sequence'] = df_filtered.apply(get_centered_sequence, axis=1)

def reverse_complement(sequence):
    # Dictionnaire de complémentarité des nucléotides
    complement = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C',
                  'a': 't', 't': 'a', 'c': 'g', 'g': 'c',
                  'N': 'N', 'n': 'n'}  # 'N' reste 'N', 'n' reste 'n'
    
    # Générer la séquence complémentaire
    complement_sequence = ''.join(complement.get(base, base) for base in sequence)
    
    # Inverser la séquence complémentaire
    reverse_complement_sequence = complement_sequence[::-1]
    
    return reverse_complement_sequence

df_filtered['centred_sequence_'] = df_filtered.apply(lambda row: reverse_complement(row['centred_sequence'].upper()) if row['strand'] == '-' else row['centred_sequence'], axis=1)
df_filtered.drop(columns = ["centred_sequence"], inplace = True)
df_filtered.rename(columns = {"centred_sequence_" : "centred_sequence"}, inplace = True)

NC_000001.11


/var/tmp/ipykernel_7402/1252905593.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['centred_sequence'] = df_filtered.apply(get_centered_sequence, axis=1)
/var/tmp/ipykernel_7402/1252905593.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['centred_sequence_'] = df_filtered.apply(lambda row: reverse_complement(row['centred_sequence'].upper()) if row['strand'] == '-' else row['centred_sequence'], axis=1)
/var/tmp/ipykernel_7402/1252905593.py:77: SettingWithCopyWarning: 
A value

## Function to create the directory 

In [13]:
import os
import shutil

def create_or_empty_directory(dir_path):
    if os.path.exists(dir_path):
        # If directory exists, delete all its contents
        shutil.rmtree(dir_path)
    # Create the directory (or recreate it if it was deleted)
    os.makedirs(dir_path)


## Create the train test set similar as dexter 

In [14]:
if sequence_length_left > 0 and sequence_length_right > 0:
    path_ = f"DNABERT_2/DATASETS/GRCH38/{sequence_length_left}_tss_{sequence_length_right}"
elif sequence_length_left > 0:
    path_ = f"DNABERT_2/DATASETS/GRCH38/{sequence_length_left}_tss"
elif sequence_length_right > 0:
    path_ = f"DNABERT_2/DATASETS/GRCH38/tss_{sequence_length_right}"
else:
    path_ = "DNABERT_2/DATASETS/GRCH38/tss"
create_or_empty_directory(path_)


import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the initial DataFrame
pituitary = pd.read_csv("human_genes_expression_pit/created_by_me/final_result.csv")
final_df = pd.merge(df_filtered, pituitary, left_on="GENE_ID", right_on='gene_id')

# Replace zeros with a small value to avoid errors in log transformation
final_df['Pituitary'] = final_df['Pituitary'].replace(0, 0.001)

# Take the logarithm base 10 of the 'Pituitary' column
final_df['Pituitary'] = np.log10(final_df['Pituitary'])

# Select desired columns
final_df = final_df[['centred_sequence', 'Pituitary', 'last_gene_id', 'GENE_ID', 'dex_gene_id']]

# Convert 'centred_sequence' to uppercase
final_df['centred_sequence'] = final_df['centred_sequence'].str.upper()

# Load the training set data
train_dex = pd.read_csv("DExTER/example/my_experience_human/training_set.log", names=["gene_id_dex_version"])

# Merge with final_df to get training data with required columns
data_train_dex = pd.merge(train_dex, final_df, how="left", left_on="gene_id_dex_version", right_on="dex_gene_id")

# Select specific columns for training data
data_train_dex = data_train_dex[['centred_sequence', 'Pituitary', 'last_gene_id']]

# Remove rows with NaN values (if any)
data_train_dex = data_train_dex.dropna()

# Splitting into train (87%) and dev (13%)
train_df, dev_df = train_test_split(data_train_dex, test_size=0.13, random_state=42)


# Save train and dev datasets to CSV files
train_df.to_csv(path_ + "/train.csv", index=False)
dev_df.to_csv(path_ + "/dev.csv", index=False)

# Load the testing set data
test_dex = pd.read_csv("DExTER/example/my_experience_human/testing_set.log", names=["gene_id_dex_version"])

# Merge with final_df to get testing data with required columns
data_test_dex = pd.merge(test_dex, final_df, how="left", left_on="gene_id_dex_version", right_on="dex_gene_id")

# Select specific columns for testing data
data_test_dex = data_test_dex[['centred_sequence', 'Pituitary', 'last_gene_id']]

# Remove rows with NaN values (if any)
data_test_dex = data_test_dex.dropna()

# Save testing dataset to a CSV file
data_test_dex.to_csv(path_ + "/test.csv", index=False)


## datasets with differents seeds as dexter

In [30]:
import pandas as pd
#modify here the path where you want your files 
path_ = "DNABERT_2/DATASETS/GRCH38/2000tss2000_seed4"

final_df = pd.read_csv("DNABERT_2/DATASETS/GRCH38/2000tss2000/2000_tss_2000.csv")

#modify here the dexter files 
train_dex = pd.read_csv("DExTER/example/grch38_SEED4/training_set.log", names=["gene_id_dex_version"])
test_dex = pd.read_csv("DExTER/example/grch38_SEED4/testing_set.log", names=["gene_id_dex_version"])

# Remove version numbers from 'gene_id' columns
final_df['gene_id_stripped'] = final_df['last_gene_id'].apply(lambda x: x.split('.')[0])
train_dex['gene_id_stripped'] = train_dex['gene_id_dex_version'].apply(lambda x: x.split('.')[0])
test_dex['gene_id_stripped'] = test_dex['gene_id_dex_version'].apply(lambda x: x.split('.')[0])



create_or_empty_directory(path_)


import numpy as np
from sklearn.model_selection import train_test_split


# Merge with final_df to get training data with required columns
data_train_dex = pd.merge(train_dex, final_df, how="left", left_on="gene_id_stripped", right_on="gene_id_stripped")

# Select specific columns for training data
data_train_dex = data_train_dex[['centred_sequence', 'Pituitary', 'last_gene_id']]

# Remove rows with NaN values (if any)
data_train_dex = data_train_dex.dropna()

# Splitting into train (87%) and dev (13%)
train_df, dev_df = train_test_split(data_train_dex, test_size=0.13, random_state=42)


# Save train and dev datasets to CSV files
train_df.to_csv(path_ + "/train.csv", index=False)
dev_df.to_csv(path_ + "/dev.csv", index=False)


# Merge with final_df to get testing data with required columns
data_test_dex = pd.merge(test_dex, final_df, how="left", left_on="gene_id_stripped", right_on="gene_id_stripped")

# Select specific columns for testing data
data_test_dex = data_test_dex[['centred_sequence', 'Pituitary', 'last_gene_id']]

# Remove rows with NaN values (if any)
data_test_dex = data_test_dex.dropna()

# Save testing dataset to a CSV file
data_test_dex.to_csv(path_ + "/test.csv", index=False)
